In [1]:
# Env name and device
env_name = 'maze2d-open-v1'
horizon = 64
device = 0

In [2]:
# get dataset
from diffuser.datasets import GoalDataset, UnconditionedDataset

dataset = UnconditionedDataset(env=env_name, termination_penalty=None, 
                      horizon=horizon,
                      normalizer='LimitsNormalizer', 
                      preprocess_fns=[],
                      use_padding=False,
                      max_path_length=40000
                      )

pybullet build time: May 20 2022 19:45:31
load datafile: 100%|██████████| 8/8 [00:00<00:00, 33.59it/s]


[ datasets/buffer ] Finalized replay buffer | 5000 episodes
[ datasets/buffer ] Fields:
    actions: (5000, 40000, 2)
    info/goal: (5000, 40000, 2)
    info/qpos: (5000, 40000, 2)
    info/qvel: (5000, 40000, 2)
    observations: (5000, 40000, 4)
    rewards: (5000, 40000, 1)
    terminals: (5000, 40000, 1)
    timeouts: (5000, 40000, 1)
    next_observations: (5000, 40000, 4)
    normed_observations: (5000, 40000, 4)
    normed_actions: (5000, 40000, 2)


In [3]:
# Get renderer
from diffuser.utils.rendering import Maze2dRenderer

renderer = Maze2dRenderer(env_name)

/home/cspark/diffuser/diffuser/utils/config.py:21: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  class Config(collections.Mapping):


In [4]:
# Get model
from diffuser.models import TemporalUnet

observation_dim = dataset.observation_dim
action_dim = dataset.action_dim

model = TemporalUnet(horizon=horizon,
                     transition_dim=observation_dim + action_dim,
                     cond_dim=observation_dim,
                     dim_mults=(1, 4, 8),
                     )
model.to(device=device);

[ models/temporal ] Channel dimensions: [(6, 32), (32, 128), (128, 256)]
[(6, 32), (32, 128), (128, 256)]


In [5]:
# Get diffusion model
from diffuser.models import GaussianDiffusion

diffusion_model = GaussianDiffusion(
    model=model,
    horizon=horizon,
    observation_dim=observation_dim,
    action_dim=action_dim,
    n_timesteps=64,
    loss_type='l2',
    clip_denoised=True,
    predict_epsilon=False,
    action_weight=1,
    loss_weights=None,
    loss_discount=1,
)
diffusion_model.to(device=device);

In [6]:
# Load trained model
import torch

modelpath = f"../logs/maze2d-open-v1/wo_conditon_H64/state_160000.pt"
diffusion_model.load_state_dict(torch.load(modelpath)['ema'])
diffusion_model.eval()

GaussianDiffusion(
  (model): TemporalUnet(
    (time_mlp): Sequential(
      (0): SinusoidalPosEmb()
      (1): Linear(in_features=32, out_features=128, bias=True)
      (2): Mish()
      (3): Linear(in_features=128, out_features=32, bias=True)
    )
    (downs): ModuleList(
      (0): ModuleList(
        (0): ResidualTemporalBlock(
          (blocks): ModuleList(
            (0): Conv1dBlock(
              (block): Sequential(
                (0): Conv1d(6, 32, kernel_size=(5,), stride=(1,), padding=(2,))
                (1): Rearrange('batch channels horizon -> batch channels 1 horizon')
                (2): GroupNorm(8, 32, eps=1e-05, affine=True)
                (3): Rearrange('batch channels 1 horizon -> batch channels horizon')
                (4): Mish()
              )
            )
            (1): Conv1dBlock(
              (block): Sequential(
                (0): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,))
                (1): Rearrange('batch channels hori

In [7]:
# get condition
import einops
from diffuser.utils import to_device, apply_dict

n_samples = 5

dataloader = torch.utils.data.DataLoader(
            dataset, batch_size=1, num_workers=0, shuffle=True, pin_memory=True
        )

In [ ]:
datas = []

for i in range(2):
    data = next(iter(dataloader))
    
    if i == 0:
        while 0 < data.conditions[0][0, 0]:
            data = next(iter(dataloader))
        
    else:
        while 0 > data.conditions[0][0, 0]:
            data = next(iter(dataloader))
    datas.append(data)



In [8]:
# conditions = to_device({0:data.conditions[0]})
data = next(iter(dataloader))
conditions = to_device(data.conditions)
conditions = apply_dict(
                einops.repeat,
                conditions,
                'b d -> (repeat b) d', repeat=n_samples,
            )
conditions = {}

# conditions_lst = []

# for data in datas:
#     conditions = to_device(data.conditions)
#     conditions_lst.append(conditions)

# conditions = conditions_lst[0]
# for condition_element in conditions_lst[1:]:
#     for k, v in condition_element.items():
#         conditions[k] = torch.cat([conditions[k], v], dim=0)

# conditions[0] = torch.cat([conditions[0], conditions[0]], dim=0)
# keys_lst = list(conditions.keys())
# conditions[keys_lst[-1]] = torch.cat([conditions[keys_lst[-1]], conditions[keys_lst[-1]].flip(dims=(0,))], dim=0)

# conditions = apply_dict(
#                 einops.repeat,
#                 conditions,
#                 'b d -> (b copy) d', copy=n_samples,
#             )

NameError: name 'data' is not defined

In [ ]:
conditions

In [ ]:
# sample trajectory 
import numpy as np
from diffuser.utils import to_np

## [ n_samples x horizon x (action_dim + observation_dim) ]
samples = diffusion_model.conditional_sample(conditions, verbose=False, batch_size=n_samples*20)
samples = to_np(samples)

## [ n_samples x horizon x observation_dim ]
normed_observations = samples[:, :, action_dim:]

# [ 1 x 1 x observation_dim ]
# normed_conditions = to_np(conditions[0])[:,None]

# from diffusion.datasets.preprocessing import blocks_cumsum_quat
# observations = conditions + blocks_cumsum_quat(deltas)
# observations = conditions + deltas.cumsum(axis=1)

## [ n_samples x (horizon + 1) x observation_dim ]
normed_observations = np.concatenate([
    # normed_conditions,
    normed_observations
], axis=1)

## [ n_samples x (horizon + 1) x observation_dim ]
observations = dataset.normalizer.unnormalize(normed_observations, 'observations')

In [ ]:
renderer.composite(f"dummy.png", observations, ncol=n_samples)